# Pairing Optimization

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
from po.data.readingdata import file_path_finder
from po.features.pairing_optimization import po
from po.features.pairing_optimization import max_streak

In [3]:
date = datetime.now().strftime("%m%d%Y_%H%M")
date

'05042024_2133'

## Load Data

In [4]:
df_path = file_path_finder('*merchandise*')

columns_to_read = ['Brand', 'Model', 'Description', 'ColorName','Gender','Size', 'Type','Active', 'On Hand']
df = pd.read_excel(df_path, usecols=columns_to_read)

df.head(1)

,Brand,Model,Size,Type,Description,Active,ColorName,Gender,On Hand
0,Brand 1,Model 1,20,PANT,description 1,NOT MAKEABLE,color 1,,0


## Clean data

Gender data is not standarized

In [5]:
# copy data frame to keep original data
df_clean = df.copy()

In [6]:
# Dictionaries to replace values / normalized categories
replace_dict = {"MEN'S":'MEN',
"GIRL'S":'GIRL',
"JUNIOR'S":'JUNIOR',
"  WOMEN":'WOMEN',
"  UNISEX":'UNISEX',
"  MEN":'MEN',
"  CHILD UNISEX":'CHILD UNISEX',
" WOMEN":'WOMEN',
"BOY'S":'BOY',
"Women":'WOMEN',
"WOMEN PETITE":'WOMEN',
"UNISEX                  MENWOMEN":'UNISEX',
"Unisex":'UNISEX',
"Men":'MEN',
"Girl":'GIRL',
'UNISEX                        (MEN WOMEN':'UNISEX',
'UNISEX                        MEN WOMEN':'UNISEX',
'UNISEX SHORT               (MEN WOMEN ME':'UNISEX'}

In [7]:
df_clean['Gender'] = df_clean['Gender'].replace(replace_dict)

### Prepare Data for Item Pairing Optimization

#### Defining universe of items to pair
---

Focus in creating sets from 2 types of products ( `top` and `pants`) from excess inventory
>🌟 This will help reduce obsolete inventory to create a new products

In [8]:
df_clean = df_clean.loc[df_clean['Type'].isin(['PANT', 'TOP']) & (df_clean['Active'] == 'NOT MAKEABLE')]

## Pairing Optimization 🔢

In [9]:
# Define constraints for pairing
constraints = ['Brand','ColorName','Gender','Size']

# Components of mataching
components = ['Type']

# resource
resource = ['On Hand']

Results ✅

In [11]:
# run pairing optimization
df_group_min = po(df_clean, constraints, components, resource)
print("Results..")
print(df_group_min.head())

Results..
     Brand   ColorName Gender Size  On Hand
0  Brand 1     color 1  WOMEN    L      2.0
1  Brand 1     color 1  WOMEN   XL      1.0
2  Brand 1  color 1094  WOMEN    S      2.0
3  Brand 1   color 111  WOMEN    S      7.0
4  Brand 1     color 3  WOMEN  2XL      1.0


## Consecutive event detection 🔥

### Prepare data for streak calculation by making sure sizes are available for all products

In [12]:
# shape data to bring over the full range of sizes
df_group_min_pt = pd.pivot_table(df_group_min,index=['Brand','ColorName','Gender'], columns=['Size'], fill_value = 0)
# drop level
df_group_min_pt.columns = df_group_min_pt.columns.droplevel(0)

# sort the sizes range
sizes = ['XXS', 'XS', 'S', 'M', 'L', 'XL', '2XL', '3XL', '4XL', '5XL']
df_group_min_pt = df_group_min_pt[sizes]

# calculate total number of sets accross all sizes
df_group_min_pt['Total'] = df_group_min_pt.sum(axis=1).values

# remove products no pairs accross all sizes
df_group_min_pt = df_group_min_pt[df_group_min_pt['Total']!=0]
# remove size hiarchy
df_group_min_pt.reset_index(inplace=True)

# create id to pair with other tables
df_group_min_pt['ID'] = df_group_min_pt['Brand'].str.strip() + df_group_min_pt['ColorName'].str.strip() + df_group_min_pt['Gender'].str.strip()
cols = ['ID','Brand', 'ColorName', 'Gender',
                                    'XXS',
                                    'XS',
                                    'S',
                                    'M',
                                    'L',
                                    'XL',
                                    '2XL',
                                    '3XL',
                                    '4XL',
                                    '5XL',
                                    'Total']
df_group_min_pt = df_group_min_pt[cols]
df_group_min_pt.head(5)

Size,ID,Brand,ColorName,Gender,XXS,XS,S,M,L,XL,2XL,3XL,4XL,5XL,Total
0,Brand 1color 1WOMEN,Brand 1,color 1,WOMEN,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0
1,Brand 1color 1094WOMEN,Brand 1,color 1094,WOMEN,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,Brand 1color 111WOMEN,Brand 1,color 111,WOMEN,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
3,Brand 1color 3WOMEN,Brand 1,color 3,WOMEN,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0
4,Brand 1color 4WOMEN,Brand 1,color 4,WOMEN,0.0,0.0,0.0,0.0,4.0,9.0,4.0,0.0,0.0,0.0,17.0


### Find the number of continous sets for each product combination

In [13]:
df_group_min_pt = max_streak(df_group_min_pt, 4, 14)
df_group_min_pt.head(5)

Size,ID,Brand,ColorName,Gender,XXS,XS,S,M,L,XL,2XL,3XL,4XL,5XL,Total,maxcontinous
0,Brand 1color 1WOMEN,Brand 1,color 1,WOMEN,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,2
1,Brand 1color 1094WOMEN,Brand 1,color 1094,WOMEN,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1
2,Brand 1color 111WOMEN,Brand 1,color 111,WOMEN,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1
3,Brand 1color 3WOMEN,Brand 1,color 3,WOMEN,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,1
4,Brand 1color 4WOMEN,Brand 1,color 4,WOMEN,0.0,0.0,0.0,0.0,4.0,9.0,4.0,0.0,0.0,0.0,17.0,3


## Identify the products components to build pairs 🧺

In [14]:
# id to blend data to paired df
df_clean['ID'] = df_clean['Brand'].str.strip() + df_clean['ColorName'].str.strip() + df_clean['Gender'].str.strip()

In [15]:
# filter for pairing compatible products
df_clean_set_components = df_clean.loc[df_clean['ID'].isin(df_group_min_pt['ID'])]

In [16]:
df_clean_set_components.head(5)

,Brand,Model,Size,Type,Description,Active,ColorName,Gender,On Hand,ID
4,Brand 1,Model 1,10,PANT,description 1,NOT MAKEABLE,color 4,WOMEN,0,Brand 1color 4WOMEN
5,Brand 1,Model 1,12,PANT,description 1,NOT MAKEABLE,color 4,WOMEN,0,Brand 1color 4WOMEN
6,Brand 1,Model 1,14,PANT,description 1,NOT MAKEABLE,color 4,WOMEN,0,Brand 1color 4WOMEN
7,Brand 1,Model 1,16,PANT,description 1,NOT MAKEABLE,color 4,WOMEN,0,Brand 1color 4WOMEN
8,Brand 1,Model 1,8,PANT,description 1,NOT MAKEABLE,color 4,WOMEN,0,Brand 1color 4WOMEN


In [17]:
# pivot to have get the same view as the assortment dataset
df_f = pd.pivot_table(df_clean_set_components, index=['Brand', 'Model', 'Type', 'Description', 'Active', 'ColorName',
       'Gender', 'ID'], columns=['Size'], aggfunc='sum')
# clean and process dataframe
df_f.columns = df_f.columns.droplevel(0)
df_f = df_f.loc[:,sizes]
df_f['Total'] = df_f.sum(axis=1).values
df_f = df_f[df_f['Total']>0]

df_f.reset_index(inplace=True)

Dataset with pairs and a column that helps determine continous sizes ✅

In [18]:
df_f

Size,Brand,Model,Type,Description,Active,ColorName,Gender,ID,XXS,XS,S,M,L,XL,2XL,3XL,4XL,5XL,Total
0,Brand 1,Model 1183,TOP,description 431,NOT MAKEABLE,color 1,WOMEN,Brand 1color 1WOMEN,NaN,112.0,169.0,189.0,103.0,76.0,0.0,NaN,NaN,NaN,649.0
1,Brand 1,Model 2225,TOP,description 158,NOT MAKEABLE,color 1094,WOMEN,Brand 1color 1094WOMEN,NaN,0.0,3.0,4.0,1.0,4.0,0.0,0.0,NaN,NaN,12.0
2,Brand 1,Model 2225,TOP,description 158,NOT MAKEABLE,color 3,WOMEN,Brand 1color 3WOMEN,0.0,0.0,5.0,8.0,5.0,3.0,3.0,0.0,0.0,0.0,24.0
3,Brand 1,Model 2225,TOP,description 158,NOT MAKEABLE,color 4,WOMEN,Brand 1color 4WOMEN,NaN,0.0,6.0,15.0,17.0,9.0,4.0,0.0,0.0,0.0,51.0
4,Brand 1,Model 2225,TOP,description 158,NOT MAKEABLE,color 5,WOMEN,Brand 1color 5WOMEN,0.0,0.0,5.0,28.0,23.0,4.0,1.0,0.0,0.0,0.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,Brand 9,Model 3822,PANT,description 999,NOT MAKEABLE,color 5953,WOMEN,Brand 9color 5953WOMEN,87.0,165.0,108.0,117.0,92.0,122.0,54.0,24.0,NaN,NaN,769.0
1527,Brand 9,Model 3823,PANT,description 999,NOT MAKEABLE,color 5953,WOMEN,Brand 9color 5953WOMEN,66.0,105.0,110.0,146.0,110.0,79.0,75.0,NaN,NaN,NaN,691.0
1528,Brand 9,Model 3836,TOP,description 139,NOT MAKEABLE,color 21,WOMEN,Brand 9color 21WOMEN,50.0,42.0,58.0,45.0,24.0,0.0,3.0,12.0,NaN,NaN,234.0
1529,Brand 9,Model 3836,TOP,description 139,NOT MAKEABLE,color 24,WOMEN,Brand 9color 24WOMEN,68.0,82.0,151.0,128.0,82.0,21.0,12.0,23.0,NaN,NaN,567.0


In [19]:
# bring over number of continous series
df_f = df_f.merge(df_group_min_pt[['ID','maxcontinous']], on='ID', how='left')
# Convert all float columns to integer
float64_columns = df_f.select_dtypes(include='float64').columns
df_f[float64_columns] = df_f[float64_columns].fillna(0)

# Now, convert float64 columns to int
df_f[float64_columns] = df_f[float64_columns].astype(int)

In [20]:
df_f = df_f[['Brand', 'Model', 'Type','ColorName',
       'Gender', 'XXS', 'XS', 'S', 'M', 'L', 'XL', '2XL', '3XL', '4XL',
       '5XL', 'Total', 'maxcontinous']]

In [21]:
df_f.sample(25).to_csv(r'..\docs\tool\results.csv', index= False)